# Assignment 8

#### 1. 复习上课内容

#### 2.理论题

##### 1.  What is semi-supervised learning ?

semi-supervised：

+ 有标签数据
$$\{(x^{r},y\_hat^{r})\}^{R}_{r=1}$$
+ 无标签数据
$$\{x^{u}\}^{R+U}_{u=R}$$
通常情况下 U>>R

有未标注数据参与训练的学习，可以分为inductive learning 和 transductive learning

+ 如果这个未标注数据同时也是测试数据，则是transductive learning
+ 如果这个未标注数据只是用于帮助训练而不用于测试的话，则是inductive learning

##### 2. What is GMM ?

高斯混合模型GMM 用来表示在总体分布（distribution）中含有 K 个子分布的概率模型

高斯混合模型的概率分布为：

$$P(x|\theta) = \Sigma^K_{k=1} \alpha_{k}\phi(x|\mu_{k},\sigma_{k})$$

K是混合模型中子高斯模型的数量，k=1,2,3...,K

$\alpha_{k}$是观测数据属于第k个子模型的概率, $\Sigma^K_{k=1} \alpha_{k} = 1$

##### 3. What are the diffreences between seed k-means and constrained k-means ?

+ Seeded K-Means:
    + Labeled data provided by user are used for initialization: initial center for cluster 𝑖 is the mean of the seed points having label 𝑖
    + Seed points are only used for initialization, and not in subsequent steps

+ Constrained K-Means:  
    + Labeled data provided by user are used to initialize K-Means algorithm.
    + Cluster labels of seed data are kept unchanged in the cluster assignment steps, and only the labels of the non-seed data are re-estimated.


+ 两个都是用有标签的数据做初始化
    + Seeded K-Means的有标签数据会和无标签数据一样进行训练更新，有可能会在之后的步骤进行一定的更新
    + Constrained K-Means会排除这些用来初始化的有标签数据(相信这些有标签的数据)。

##### 4. Please briefly describe what are self-training and co-training ? 

self-training 自我训练: 通过一系列的步骤，用已有的有标签的数据(labelled data)，去对剩下的还未标记的数据打标签。从而使得训练数据(training data)更多

+ 用有标签数据$\{(X_{1:n}, Y_{1:n})\}$进行训练
+ 用这个模型来对没有标签的数据做预测 predicr on $X_u$
+ 选取最有把握的预测结果来标记数据(unlabelled data), 把新标记好的数据加入到原来的标记好的数据集中，同时把他们从原来的数据集中删除
+ 不断重复,直到数据集不发生变化

co-training 协同学习:

假设有数据集，有多个特征，随机的把特征拆成两部分，每组数据有2个特征: x1 和 x2。其中有标签的那部分数据集为L:[x1,x2,y]，没有标签的数据集为U:[x1,x2]。

初始化数据，把L 分为L1 ([x1,y]) 和L2 ([x2,y])

重复, 直到数据集不发生变化

+ 分别用L1和L2训练出一个模型 F1和F2
+ 分别用模型F1和F2去预测U (给U打标签)（只选出最有把握的一些结果）
+ 把F1预测的top-k结果放入L2，把F2预测的top-k结果放入L1 (交叉放置)
+ 更新L 和 U，把他们从原来的数据集中删除




##### 5. What is active learning ?

让模型自动挑选出需要学习的数据
+ 从一堆不带标签的数据中
    + 主动选择一部分进行标注，而不是被动的。

+ 每标注一次，模型重新或者增量的训练一次。

+ 然后重复以上步骤

#### 3.实践题

Titanic: Machine Learning from Disaster

Data Source :https://www.kaggle.com/c/titanic/data

####  In this assignment, you have to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

低： 完成。

高：准确率 95% 以上。

In [123]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [131]:
train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')

In [132]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [133]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [134]:
# 性别对存活率的影响， 妇女小孩优先的原则，增加判断是否child的feature
print(train[['Sex','Survived']].groupby(['Sex']).mean())

        Survived
Sex             
female  0.742038
male    0.188908


In [135]:
# 船舱和fare的关系
print(train[['Pclass','Fare']].groupby(['Pclass']).mean())

             Fare
Pclass           
1       84.154687
2       20.662183
3       13.675550


### 数据预处理

In [136]:
def processing(titanic):
    #对于年龄为空的数据 填充年龄中位数
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    # child 妇女儿童优先
    titanic["child"] = titanic["Age"].apply(lambda x: 1 if x < 15 else 0)
    # sex 把性别从字符串类型转换为0或1数值型数据
    titanic["Sex"] = titanic["Sex"].apply(lambda x: 1 if x == "male" else 0)
    # familysize 家庭成员人数 兄弟姐妹父母配偶加自己
    titanic["fimalysize"] = titanic["SibSp"] + titanic["Parch"] + 1
    # 对于fare为空的数据 根据船舱等级填充fare中位数
    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())
    # 根据不同船舱等级对应的船票均值来对缺失的船票进行赋值
    fare = [0,0,0]
    for level in range(0, 3):
        fare[level] = titanic[titanic.Pclass == level + 1]['Fare'].dropna().median()
    for level in range(0, 3):
        titanic.loc[(titanic.Fare.isnull()) & (titanic.Pclass == level + 1), 'Fare'] = fare[level]
    return titanic
# 对数据进行预处理
train_data = processing(train)
test_data = processing(test)

In [138]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
child            0
fimalysize       0
dtype: int64

In [139]:
# 选取特征 Cabin字段缺失数据多
X_train = train_data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare','child','fimalysize']].as_matrix()

/Users/maytone/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [140]:
Y_train = train['Survived']

In [141]:
#训练模型
model = xgb.XGBClassifier(max_depth=2, n_estimators=20, learning_rate=0.01).fit(X_train, Y_train)

In [142]:
X_test = test_data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare','child','fimalysize']].as_matrix()

/Users/maytone/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [143]:
# 调参
from sklearn.model_selection import GridSearchCV
param_test = {
    'n_estimators':range(20,60,2),
    'max_depth': range(2, 10, 1),
    'learning_rate': list((0.1,0.05,0.01))
}
grid_search = GridSearchCV(estimator = model, param_grid = param_test, 
scoring='accuracy', cv=5)
grid_search.fit(X_train, Y_train)
grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.1, max_delta_step=0, max_depth=6,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=38, n_jobs=0, num_parallel_tree=1,
               objective='binary:logistic', random_state=0, reg_alpha=0,
               reg_lambda=1, scale_pos_weight=1, subsample=1,
               tree_method='exact', validate_parameters=1, verbosity=None),
 {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 38},
 0.8428731762065096)

In [144]:
lr = grid_search.best_params_['learning_rate']
m_depth = grid_search.best_params_['max_depth']
n_estima = grid_search.best_params_['n_estimators']

In [146]:
#评估模型
model = xgb.XGBClassifier(max_depth=m_depth, n_estimators=n_estima, learning_rate=lr).fit(X_train, Y_train)
predictions = model.predict(X_test)

# 保存预测结果
submission = pd.DataFrame({ 'PassengerId': test['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_xgboost_submission4.csv", index=False)

最终kaggle只有0.77